# Load SARS-CoV-2 Virus Strain Data from CNCB
**[Work in progress]**

This notebook downloads and standardizes viral strain and variation data from CNCB for ingestion into a Knowledge Graph.

Data source: [China National Center for Bioinformation, 2019 Novel Coronavirus Resource (2019nCoVR)](https://bigd.big.ac.cn/ncov/release_genome)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import dateutil
import re
from pathlib import Path
import glob
#from os import path
import ftplib

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
# Path will take care of handling operating system differences.
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


In [4]:
# Create a directory to cache variation data
CACHE = Path(NEO4J_HOME / 'import/cache')
CACHE.mkdir(exist_ok=True)

## Download SARS-CoV-2 Strain metadata

In [5]:
metadata_url = "https://bigd.big.ac.cn/ncov/genome/export/meta"
annotation_url = "ftp://download.big.ac.cn/GVM/Coronavirus/gff3/" 

In [6]:
df = pd.read_excel(metadata_url, dtype='str')
df.fillna('', inplace=True)

In [7]:
print("Total number of strains:", df.shape[0])

Total number of strains: 30735


In [8]:
df = df.query("`Sequence Quality` == 'High'")
df = df.query("`Nuc.Completeness` == 'Complete'")

In [9]:
print("Number of complete high quality strains", df.shape[0])

Number of complete high quality strains 14855


In [10]:
df.head()

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Wuhan Jinyintan Hospital,2020-01-19,Hubei Provincial Center for Disease Control an...
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,Bamrasnaradura Hospital,2020-01-17,"Department of Medical Sciences, Ministry of Pu..."
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,Bamrasnaradura Hospital,2020-01-17,"Department of Medical Sciences, Ministry of Pu..."
3,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01,NMDC,EPI_ISL_402120,Complete,29896,High,0/0/0/2/NO,Homo sapiens,2020-01-01,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-11,National Institute for Viral Disease Control a...
4,BetaCoV/Wuhan/IVDC-HB-01/2019,NMDC60013084-01,NMDC,EPI_ISL_402119,Complete,29891,High,0/0/0/0/NO,Homo sapiens,2019-12-30,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-10,National Institute for Viral Disease Control a...


#### Create a separate row for each Accession and Related ID

In [11]:
df['Accession ID'] = df['Accession ID'].str.strip()
df['Related ID'] = df['Related ID'].str.strip()

# combine all ids into a single column
df['alias'] = df['Accession ID'] + df['Related ID'].apply(lambda s: ',' + s if len(s) > 0 else s)
df['alias'] = df['alias'].str.replace(' ', '')

# then "explode" ids into separate rows
df['id'] = df['alias'].apply(lambda s: s.split(','))
df = df.explode('id')
df['id'] = df['id'].str.strip()
df['alias'] = df['alias'].str.replace(',', ';')

#### Assign taxonomy ids

In [12]:
# TODO a find general solution to map host name to NCBI taxonomy**

# some organism specifications are ambiguous, 
# they don't match a specificn NCBI taxonomy

taxonomy_to_id = {'Human': 'taxonomy:9606', 
                  'Homo sapiens': 'taxonomy:9606',
                  'Homo Sapiens': 'taxonomy:9606',
                  'Rhinolophus affinis': 'taxonomy:59477', 
                  'Rhinolophus malayanus': 'taxonomy:608659', 
                  'Mustela lutreola': 'taxonomy:9666',
                  'Panthera tigris jacksoni': 'taxonomy:419130',
                  'Rhinolophus sp. (bat)': 'taxonomy:49442', # ambiguous
                  'bat': 'taxonomy:49442', # ambiguous
                  'Manis javanica': 'taxonomy:9974',
                  'palm civet': 'taxonomy:71116', # ambiguous
                  'Canine': 'taxonomy:9608', # ambiguous
                  'canine': 'taxonomy:9608', # ambiguous
                  'Felis catus': 'taxonomy:9685'
                 }

In [13]:
# assign taxonomy id to host
df['hostTaxonomyId'] = df['Host'].apply(lambda s: taxonomy_to_id.get(s, s))
df['hostTaxonomyId'].unique()

array(['taxonomy:9606', 'taxonomy:59477', 'Environment', 'taxonomy:9974',
       'taxonomy:608659', 'taxonomy:419130', 'taxonomy:9666',
       'taxonomy:9608'], dtype=object)

In [14]:
df['taxonomyId'] = 'taxonomy:2697049' # SARS-CoV-2

#### Standardize node property names (CURIEs and URIs)

In [15]:
df.rename(columns={'Virus Strain Name': 'name',
                   'Sample Collection Date':'collectionDate',
                   'Location':'location'}, 
          inplace=True)

In [16]:
# https://registry.identifiers.org/registry/insdc
insdc_pattern = re.compile('^([A-Z]\d{5}|[A-Z]{2}\d{6}|[A-Z]{4}\d{8}|[A-J][A-Z]{2}\d{5})(\.\d+)?$')

In [17]:
def assign_curie(id):
    id = id.strip()
    # remove underscore to enable CURIE matching of NCBI reference sequences NC_...
    id = id.replace('NC_', 'NC') 
    if len(id) > 0:
        if id.startswith('EPI'):
            return 'https://www.gisaid.org/' + id
        elif id.startswith('NC_'):
            # NCBI reference sequences resolve with ncbiprotein CURIE
            return 'ncbiprotein:' + id
        elif insdc_pattern.match(id) != None:
            return 'insdc:' + id
        else:
            # TODO are URIs available for these cases?
            return id
    else:
        return id

In [18]:
strains = df[['id', 'name', 'alias', 'taxonomyId', 'hostTaxonomyId','collectionDate', 'location']].copy()
strains['id'] = strains['id'].apply(assign_curie)
strains.head()

,id,name,alias,taxonomyId,hostTaxonomyId,collectionDate,location
0,NMDC60013088-01,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01;EPI_ISL_402132,taxonomy:2697049,taxonomy:9606,2019-12-30,China / Hubei
0,https://www.gisaid.org/EPI_ISL_402132,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01;EPI_ISL_402132,taxonomy:2697049,taxonomy:9606,2019-12-30,China / Hubei
1,https://www.gisaid.org/EPI_ISL_403963,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,taxonomy:2697049,taxonomy:9606,2020-01-13,Thailand/ Nonthaburi Province
2,https://www.gisaid.org/EPI_ISL_403962,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,taxonomy:2697049,taxonomy:9606,2020-01-08,Thailand/ Nonthaburi Province
3,NMDC60013085-01,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01;EPI_ISL_402120,taxonomy:2697049,taxonomy:9606,2020-01-01,China / Hubei / Wuhan


In [19]:
strains.to_csv(NEO4J_HOME / "import/01e-CNCBStrain.csv", index=False)

## Merge Metadata with Variation Data

#### Get list of file names from FTP site

In [20]:
server = "download.big.ac.cn"
user = "anonymous"
password = "anonymous"
source = "/GVM/Coronavirus/gff3/"

ftp = ftplib.FTP(server)
ftp.login(user, password)
ftp.cwd(source) 
filelist=ftp.nlst()
ftp.quit()

'221 Goodbye.'

In [21]:
df_file = pd.DataFrame(filelist, columns=['filename'])

Extract identifiers from file name

Example: 2019-nCoV_CNA0013697_variants.gff3 -> CNA0013697

In [22]:
df_file['id'] = df_file['filename'].str[10:]
df_file['id'] = df_file['id'].str.replace('_variants.gff3','')

In [23]:
df_file = df_file.query("id != ''")

In [24]:
print("Number of available files:", df_file.shape[0])

Number of available files: 14711


In [25]:
df_file.head()

,filename,id
0,2019-nCoV_CNA0013697_variants.gff3,CNA0013697
1,2019-nCoV_CNA0013698_variants.gff3,CNA0013698
2,2019-nCoV_CNA0013699_variants.gff3,CNA0013699
3,2019-nCoV_CNA0013700_variants.gff3,CNA0013700
4,2019-nCoV_CNA0013701_variants.gff3,CNA0013701


In [26]:
df = df.merge(df_file, on='id')

In [27]:
print('Strains with a matching filename:', df.shape[0])

Strains with a matching filename: 14630


In [28]:
df = df.sample(n=50, random_state=5)

In [29]:
df.head()

,name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,collectionDate,location,Originating Lab,Submission Date,Submitting Lab,alias,id,hostTaxonomyId,taxonomyId,filename
10430,hCoV-19/USA/LA-SR0214/2020,EPI_ISL_437542,GISAID,,Complete,29903,High,0/0/0/9/NO,Homo Sapiens,2020-04-03,United States / Louisiana / New Orleans,Robert Garry lab,2020-05-11,Andersen lab at Scripps Research,EPI_ISL_437542,EPI_ISL_437542,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_437542_variants.gff3
553,hCoV-19/USA/WA-UW56/2020,EPI_ISL_415621,GISAID,,Complete,29828,High,0/0/0/6/NO,Homo sapiens,2020-03-09,United States / Washington / Seattle,UW Virology Lab,2020-03-18,UW Virology Lab,EPI_ISL_415621,EPI_ISL_415621,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_415621_variants.gff3
9930,hCoV-19/USA/WI-UW-304/2020,EPI_ISL_436598,GISAID,,Complete,29782,High,0/0/0/9/NO,Homo Sapiens,2020-04-13,United States / Wisconsin / Milwaukee,University of Wisconsin-Madison AIDS Vaccine R...,2020-05-06,University of Wisconsin-Madison AIDS Vaccine R...,EPI_ISL_436598,EPI_ISL_436598,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_436598_variants.gff3
4869,SARS-CoV-2/human/USA/VA_6377/2020,MT325612,GenBank,,Complete,29882,High,"0/2/0/8/18505~18506(2-2-1.00,SNP:18505; SNP:18...",Homo sapiens,2020-03-08,United States / VA,,2020-04-10,"Division of Viral Diseases, Centers for Diseas...",MT325612,MT325612,taxonomy:9606,taxonomy:2697049,2019-nCoV_MT325612_variants.gff3
13290,hCoV-19/Colombia/GVI-97769/2020,EPI_ISL_447802,GISAID,,Complete,29717,High,"0/0/0/7/28881~28883(3-3-1.00,SNP:28881; SNP:28...",Homo Sapiens,2020-04-04,Colombia / Norte de santander / Valledupar,"Instituto Nacional de Salud, Bogotá, Colombia",2020-05-18,Grupo de Investigaciones Microbiológicas-UR (G...,EPI_ISL_447802,EPI_ISL_447802,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_447802_variants.gff3


Keep only unique entries (there are a few duplicate cases)

In [30]:
df.drop_duplicates(subset='name', inplace=True)
df.drop_duplicates(subset='id', inplace=True)

In [31]:
print('Strains with a matching filename:', df.shape[0])

Strains with a matching filename: 50


#### Download variant annotations for each strain
To avoid download the same files every time, they are cached, and newly downloaded files are added to the cache.

In [32]:
names = ['taxon1', 'variantType', 'name', 'start', 'end','x1', 'x2', 'x3','taxon2', 'x4', 'strainStart', 'taxon3', 'x5', 'strainEnd', 'ref', 'alt', 'vepAnnotation']

In [33]:
def download_gff3(filename, url):
    gff3 = pd.read_csv(url, header=None, comment='#', sep='[\t|;]', engine='python', names=names)
    gff3['ref'] = gff3['ref'].str.replace('REF=','')
    gff3['alt'] = gff3['alt'].str.replace('ALT=','')
    gff3['vepAnnotation'] = gff3['vepAnnotation'].str.replace('VEP=','')
    # prepare for 3-way split (need at least two commas)
    gff3['vepAnnotation'] = gff3['vepAnnotation'].apply(lambda s: s + ',,' if s.count(',') < 2 else s)
    # 3-way split
    gff3[['variantConsequence','proteinVariant','geneVariant']] = gff3['vepAnnotation'].str.split(',', n=2, expand=True)
    gff3['geneVariant'] = gff3['geneVariant'].str.replace('gene-','')
    gff3 = gff3[['name', 'variantType', 'start', 'end', 'ref', 'alt', 'variantConsequence', 'proteinVariant', 'geneVariant']]
    
    filename = row['filename'] + '.csv'
    gff3.to_csv(CACHE / filename, index=False)

In [34]:
for index, row in df.iterrows():
    url = annotation_url + row['filename']
    filename = row['filename'] + '.csv'
    if not Path.exists(CACHE / filename):
        try:
            download_gff3(row['filename'], url)
            print(row['filename'], end=' ')
        except:
            print('Download failed for: ', row['filename'])

### Concatenate all variation data into a single dataframe

In [35]:
# use all cached data files
path = str(CACHE / '*.csv')
filenames = glob.glob(path)

# list of dataframes
dfs = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    dfs.append(df)

variations = pd.concat(dfs, axis=0, ignore_index=True)
variations.fillna('', inplace=True)

List of variant types and consequences:

https://uswest.ensembl.org/info/genome/variation/prediction/classification.html

https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html#consequences

#### Extract protein position and protein id from proteinVariant string

Example: QHD43415.1:p.5828P>L

proteinPosition: 5828
proteinId: QHD43415

In [36]:
position_pattern = re.compile(':p\.(.*?)[A-Z|\-]+')
variations['proteinPosition'] = variations['proteinVariant'].apply(
    lambda s: position_pattern.search(s).group(1) if ':p.' in s else '')

In [37]:
variations['proteinId'] = variations['proteinVariant'].apply(
    lambda s: s.split('.')[0] if '.' in s else '')

In [38]:
variations['proteinId'].unique()

array(['', 'QHD43415', 'QHD43416', 'QHD43423', 'QHD43417', 'QHD43422',
       'QHD43419', 'QHD43421', 'QHI42199', 'QHD43420', 'QHD43418'],
      dtype=object)

#### Assign SARS-CoV-2 taxonomy id

In [39]:
variations['taxonomyId'] = 'taxonomy:2697049'

#### Assign Reference genome

The first SARS-CoV-2 genome sequence is the reference for the variant annotation below.

[Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1](https://www.ncbi.nlm.nih.gov/nuccore/MN908947)

In [40]:
variations['referenceGenome'] = 'insdc:MN908947'

In [41]:
print('variantType:', variations['variantType'].unique())

variantType: ['SNP' 'Deletion' 'Indel' 'Insertion']


In [42]:
print("variantConsequence:", variations['variantConsequence'].unique())

variantConsequence: ['upstream_gene_variant' 'synonymous_variant' 'missense_variant'
 'intergenic_variant' 'downstream_gene_variant' 'coding_sequence_variant'
 'inframe_deletion' 'stop_gained' 'frameshift_variant']


In [43]:
print("Number of variants:", variations.shape[0])

Number of variants: 1712


In [44]:
variations.to_csv(NEO4J_HOME / "import/01e-CNCBVariant.csv", index=False)

In [45]:
variations.head()

,name,variantType,start,end,ref,alt,variantConsequence,proteinVariant,geneVariant,proteinPosition,proteinId,taxonomyId,referenceGenome
0,hCoV-19/Russia/Vector_88903/2020,SNP,241,241,C,T,upstream_gene_variant,DISTANCE=25,"QHD43415.1,orf1ab",,,taxonomy:2697049,insdc:MN908947
1,hCoV-19/Russia/Vector_88903/2020,SNP,3037,3037,C,T,synonymous_variant,QHD43415.1:p.924F,orf1ab:c.2772ttC>ttT,924,QHD43415,taxonomy:2697049,insdc:MN908947
2,hCoV-19/Russia/Vector_88903/2020,SNP,14408,14408,C,T,missense_variant,QHD43415.1:p.4715P>L,orf1ab:c.14144cCt>cTt,4715,QHD43415,taxonomy:2697049,insdc:MN908947
3,hCoV-19/Russia/Vector_88903/2020,SNP,23403,23403,A,G,missense_variant,QHD43416.1:p.614D>G,S:c.1841gAt>gGt,614,QHD43416,taxonomy:2697049,insdc:MN908947
4,hCoV-19/Russia/Vector_88903/2020,SNP,28881,28881,G,A,missense_variant,QHD43423.2:p.203R>K,N:c.608aGg>aAg,203,QHD43423,taxonomy:2697049,insdc:MN908947
